In [3]:
!python -m pip install pyaudio
!pip install SpeechRecognition
!pip install pyttsx3
!pip install gTTS
!pip install pydub
!pip install pyqt5
from PyQt5.QtCore import Qt

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


...


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import sys
import speech_recognition as sr
import pyttsx3
from PyQt5.QtWidgets import (
    QApplication, QWidget, QPushButton, QVBoxLayout, QLabel, QFileDialog, QScrollArea, QTextEdit
)
from PyQt5.QtCore import QThread, pyqtSignal, Qt


class SpeechToTextApp(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Speech to Text and Save as MP3")
        self.setGeometry(200, 200, 600, 500)

        # Create the main label for instructions
        self.label = QLabel("Press 'Start' to speak or upload a .wav file...", self)
        self.label.setStyleSheet("font-size: 18px; color: #000")

        # Create a QTextEdit to show the recognized speech
        self.text_display = QTextEdit(self)
        self.text_display.setReadOnly(True)
        self.text_display.setStyleSheet("font-size: 14px; color: #000;")

        # Create a scroll area to allow scrolling for large text
        self.scroll_area = QScrollArea(self)
        self.scroll_area.setWidget(self.text_display)
        self.scroll_area.setWidgetResizable(True)

        # Buttons
        self.start_button = QPushButton('Start', self)
        self.start_button.clicked.connect(self.start_speech_recognition)

        self.stop_button = QPushButton('Stop', self)
        self.stop_button.clicked.connect(self.stop_speech_recognition)
        self.stop_button.setEnabled(False)

        self.upload_button = QPushButton('Upload WAV File', self)
        self.upload_button.clicked.connect(self.process_wav_file)

        self.save_button = QPushButton('Save as MP3', self)
        self.save_button.clicked.connect(self.save_as_mp3)
        self.save_button.setEnabled(False)

        self.clear_button = QPushButton('Clear Chat', self)
        self.clear_button.clicked.connect(self.clear_chat)

        self.exit_button = QPushButton('Exit', self)
        self.exit_button.clicked.connect(self.close)

        # Layout
        self.layout = QVBoxLayout(self)
        self.layout.addWidget(self.label)
        self.layout.addWidget(self.scroll_area)
        self.layout.addWidget(self.start_button)
        self.layout.addWidget(self.stop_button)
        self.layout.addWidget(self.upload_button)
        self.layout.addWidget(self.save_button)
        self.layout.addWidget(self.clear_button)
        self.layout.addWidget(self.exit_button)

        self.speech_thread = None
        self.recognized_text = ""
        self.is_listening = False

        # Ensure the default window behavior
        self.setAttribute(Qt.WA_DeleteOnClose)

    def start_speech_recognition(self):
        self.is_listening = True
        self.label.setText("Listening... Please speak now!")
        self.start_button.setEnabled(False)
        self.stop_button.setEnabled(True)

        self.speech_thread = SpeechRecognitionThread()
        self.speech_thread.result_signal.connect(self.display_recognized_text)
        self.speech_thread.start()

    def stop_speech_recognition(self):
        self.is_listening = False
        if self.speech_thread:
            self.speech_thread.stop_listening()
        self.label.setText("Stopped listening. Save or clear text as needed.")
        self.save_button.setEnabled(True)
        self.stop_button.setEnabled(False)
        self.start_button.setEnabled(True)

    def display_recognized_text(self, text):
        if self.is_listening:
            self.recognized_text += " " + text
            self.text_display.setText(self.recognized_text)
            self.text_display.verticalScrollBar().setValue(
                self.text_display.verticalScrollBar().maximum()
            )

    def process_wav_file(self):
        file_name, _ = QFileDialog.getOpenFileName(self, "Open WAV File", "", "WAV Files (*.wav)")
        if file_name:
            recognizer = sr.Recognizer()
            try:
                with sr.AudioFile(file_name) as source:
                    recognizer.adjust_for_ambient_noise(source, duration=2)
                    audio = recognizer.record(source)
                    text = recognizer.recognize_google(audio)
                    self.recognized_text += " " + text
                    self.text_display.setText(self.recognized_text)
                    self.text_display.verticalScrollBar().setValue(
                        self.text_display.verticalScrollBar().maximum()
                    )
                    self.label.setText("Text extracted from WAV file.")
                    self.save_button.setEnabled(True)
            except sr.UnknownValueError:
                self.label.setText("Could not understand audio in the WAV file.")
            except Exception as e:
                self.label.setText(f"Error processing WAV file: {e}")

    def save_as_mp3(self):
        if self.recognized_text.strip():
            file_name, _ = QFileDialog.getSaveFileName(self, "Save as MP3", "", "MP3 Files (*.mp3)")
            if file_name:
                self.text_to_mp3(self.recognized_text.strip(), file_name)
                self.label.setText(f"Saved as MP3: {file_name}")
 
    def text_to_mp3(self, text, file_name):
        engine = pyttsx3.init()

        # Slow down speech rate
        rate = engine.getProperty('rate')
        engine.setProperty('rate', rate - 50)

        # Set female voice
        voices = engine.getProperty('voices')
        for voice in voices:
            if 'female' in voice.name.lower():
                engine.setProperty('voice', voice.id)
                break

        engine.save_to_file(text, file_name)
        engine.runAndWait()

    def clear_chat(self):
        self.recognized_text = ""
        self.text_display.clear()
        self.label.setText("Press 'Start' to speak or upload a .wav file...")

    def closeEvent(self, event):
        if self.speech_thread:
            self.speech_thread.stop_listening()
        event.accept()


class SpeechRecognitionThread(QThread):
    result_signal = pyqtSignal(str)

    def __init__(self):
        super().__init__()
        self.running = True

    def run(self):
        recognizer = sr.Recognizer()
        mic = sr.Microphone()

        try:
            with mic as source:
                recognizer.adjust_for_ambient_noise(source, duration=2)
                recognizer.dynamic_energy_threshold = True

                while self.running:
                    try:
                        print("Listening...")
                        audio = recognizer.listen(source, timeout=10, phrase_time_limit=10)
                        recognized_text = recognizer.recognize_google(audio)
                        print(f"Recognized Text: {recognized_text}")  # Debugging
                        self.result_signal.emit(recognized_text)
                    except sr.WaitTimeoutError:
                        print("Timeout: No speech detected.")  # Debugging
                        continue
                    except sr.UnknownValueError:
                        print("Speech recognition could not understand the audio.")  # Debugging
                        continue
                    except Exception as e:
                        print(f"Error: {e}")  # Debugging
        except Exception as e:
            print(f"Microphone setup failed: {e}")

    def stop_listening(self):
        self.running = False


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = SpeechToTextApp()
    window.show()
    sys.exit(app.exec_())

Listening...
Speech recognition could not understand the audio.
Listening...
Speech recognition could not understand the audio.
Listening...
Speech recognition could not understand the audio.
Listening...
Recognized Text: hello can you hear me
Listening...
Recognized Text: how r u
Listening...
Recognized Text: are you ok
Listening...
Recognized Text: Giriraj Pushpa
Listening...
Recognized Text: Giriraj Giriraj
Listening...
Speech recognition could not understand the audio.
Listening...
Recognized Text: Giriraj Pushpa
Listening...
Speech recognition could not understand the audio.
Listening...
Speech recognition could not understand the audio.
Listening...
Recognized Text: did you watch Pushpa
Listening...
Speech recognition could not understand the audio.
Listening...
Timeout: No speech detected.
Listening...
